In [ ]:
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C
from stable_baselines.common.env_checker import check_env

In [ ]:
import gym
import w_mac
from collections import defaultdict
import matplotlib as plt
import networkx as nx
import dill
from ray import tune
from ray.tune.suggest.ax import AxSearch

In [ ]:
import os
import argparse
import logging
import dill
import networkx as nx
from copy import deepcopy
from ax.service.ax_client import AxClient
import ray
from ray import tune
from ray.tune import report
from ray.tune.suggest.ax import AxSearch
# from flowsim.environment.network import Network
# from flowsim.environment.hopcount_env import HopCountEnv
# from flowsim.environment.delay_env import NetworkDelayEnv
# from flowsim.tuning.monitor import OptimizationCallback
from stable_baselines import A2C
from stable_baselines.common.env_checker import check_env
# from stable_baselines.common.monitor import Monitor
from stable_baselines.common.callbacks import EveryNTimesteps
from w_mac.envs.w_mac_env import W_MAC_Env
# from w_mac.envs.monitor import OptimizationCallback
from stable_baselines.common.evaluation import evaluate_policy
from ray.tune.registry import register_env

In [ ]:
env = gym.make('wmac-tune-v0') #env without passing graph
# env.reset()

In [ ]:
# check_env(env)

In [ ]:
assert(dill.pickles(env))

In [ ]:
# error - TypeError: __init__() takes 1 positional argument but 2 were given


# config = {
#     "env": deepcopy(W_MAC_Env),
#     "num_workers": 2,
#     # "vf_share_layers": tune.grid_search([True, False]),
#     "lr": tune.grid_search([1e-4, 1e-5, 1e-6]),
#     }
# register_env("testenv", lambda config: TestEnv(config)

# results = tune.run(
#     'A2C', 
#     stop={
#         'timesteps_total': 100000
#     },
#     config=config)


ray.shutdown()
ray.init()
register_env("wmac-tune-v0", lambda config: W_MAC_Env)
tune.run(
    "A2C",
    config={
        "env": 'wmac-tune-v0'
        # "num_workers": 2
    }
)

In [ ]:
ray.init(ignore_reinit_error=True)
register_env("wmac-tune-v0", lambda config: W_MAC_Env(config))
config = {
    "env": 'wmac-tune-v0',
    # "num_workers": 2,
    # "vf_share_layers": tune.grid_search([True, False]),
    "lr": tune.grid_search([1e-4, 1e-5, 1e-6]),
    }

stop = {'timesteps_total': 10000
}
results = tune.run(
    'A2C', 
    
    config=config,
    stop = stop
    )

In [ ]:
dill.pickles(env)

In [ ]:
RAY_TUNE_SAMPLES = 48
EVAL_EPISODES = 5
TOTAL_TIMESTEPS = 10000

In [ ]:
model = W_MAC_Env()

In [ ]:
# haitham's example, unregister env error
ray.shutdown()
ray.init()
tune.run(
    "A2C",
    stop={"episode_reward_mean": 200},
    config={
        "env": 'wmac-tune-v0',
        "num_gpus": 0,
        # "num_workers": 1,
        "lr": tune.grid_search([0.01, 0.001, 0.0001]),
    },)


In [ ]:
#without search agent, pickling error
def evaluate_objective(config):
    tune_env = deepcopy(model)
    
    tune_agent = A2C("MlpPolicy", tune_env, **config)
    tune_agent.learn(total_timesteps=1000)
    episode_rewards, episode_lengths = evaluate_policy(tune_agent, tune_env,
                                                           5,
                                                           render=False,
                                                           deterministic=deterministic,
                                                           return_episode_rewards=True)
    episode_reward_mean, std_reward = np.mean(
            episode_rewards), np.std(episode_rewards)
    mean_ep_length, std_ep_length = np.mean(
            episode_lengths), np.std(episode_lengths)
    
    tune.report(episode_reward_mean=episode_reward_mean,
            std_reward=std_reward,
            mean_ep_length=mean_ep_length,
            std_ep_length=std_ep_length)

analysis = tune.run(evaluate_objective, metric='episode_reward_mean', mode='max')
best_trial = analysis.best_trial
best_config = analysis.best_config
print(best_trial, best_config)

   


In [ ]:
from stable_baselines.common.evaluation import evaluate_policy


In [ ]:
#register env and tune run
tune.register_trainable("wmac-tune-v0", w_mac.envs.w_mac_env.W_MAC_Env)
ray.init()

tune.run_experiments({
    "my_experiment": {
        "run": 'wmac-tune-v0',
        "stop": { "mean_accuracy": 100 },
        "config": {
            "alpha": tune.grid_search([0.2, 0.4, 0.6]),
            "beta": tune.grid_search([1, 2]),
        }
    }
})